#Imports

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving saude_do_sono_estilo_vida.csv to saude_do_sono_estilo_vida.csv


In [ ]:
import io
import pandas as pd
import numpy as np

df = pd.read_csv(io.BytesIO(uploaded['saude_do_sono_estilo_vida.csv']))

#1
##Mudar nomes de colunas específicas

In [ ]:
df = df.rename(columns={
    'ID': 'Identificador',
    'Pressão sanguíneaaaa': 'Pressão sanguínea',
    'Ocupação': 'Profissão',
    'Categoria BMI': 'Categoria IMC'
})

df.head()

,Identificador,Gênero,Idade,Profissão,Duração do sono,Qualidade do sono,Nível de atividade física,Nível de estresse,Categoria IMC,Pressão sanguínea,Frequência cardíaca,Passos diários,Distúrbio do sono
0,1,Homem,27,Eng. de Software,6.1,6,42,6,Sobrepeso,126/83,77,4200,Nenhuma
1,2,Homem,28,Médico(a),6.2,6,60,8,Normal,125/80,75,10000,Nenhuma
2,3,Homem,28,Médico(a),6.2,6,60,8,Normal,125/80,75,10000,Nenhuma
3,4,Homem,28,Representante de Vendas,5.9,4,30,8,Obesidade,140/90,85,3000,Apneia do sono
4,5,Homem,28,Representante de Vendas,5.9,4,30,8,Obesidade,140/90,85,3000,Apneia do sono


#2
##Média, moda e mediana

In [ ]:
group_up = df.groupby('Profissão')['Duração do sono']

mean_group = group_up.mean().apply(lambda x: f'{x:.2f}')
mode_group = group_up.apply(lambda x: f'{x.mode()[0]:.2f}')
median_group = group_up.apply(np.median)

results = pd.DataFrame({
    'Mean': mean_group,
    'Mode': mode_group,
    'Median': median_group
})

results

,Mean,Mode,Median
Profissão,,,
Advogado(a),7.41,7.20,7.3
Cientista,6.00,5.80,6.0
Contador(a),7.11,7.20,7.2
Enfermeiro(a),7.05,6.10,6.5
Eng. de Software,6.75,7.50,6.8
Engenheiro(a),7.99,8.40,8.3
Gerente,6.90,6.90,6.9
Médico(a),6.97,6.00,7.6
Pessoa Vendendora,6.40,6.50,6.4


In [ ]:
results_agg = group_up.agg(
    Mean = lambda x: f'{x.mean():.2f}',
    Mode = lambda x: f'{x.mode()[0]:.2f}',
    Median = lambda x: f'{np.median(x):.2f}'
)

results_agg

,Mean,Mode,Median
Profissão,,,
Advogado(a),7.41,7.20,7.30
Cientista,6.00,5.80,6.00
Contador(a),7.11,7.20,7.20
Enfermeiro(a),7.05,6.10,6.50
Eng. de Software,6.75,7.50,6.80
Engenheiro(a),7.99,8.40,8.30
Gerente,6.90,6.90,6.90
Médico(a),6.97,6.00,7.60
Pessoa Vendendora,6.40,6.50,6.40


#3
##Obesos em engenharia de software

In [ ]:
software_engineer = df[df['Profissão'] == 'Eng. de Software']

obese = len(software_engineer[software_engineer['Categoria IMC'] == 'Obesidade'])
total_people = len(software_engineer)
percentage_obese = (obese / total_people) * 100

print(f'Porcentagem de obesos em engenheiros de software: {percentage_obese:.2f}%')

Porcentagem de obesos em engenheiros de software: 25.00%


In [ ]:
bmi_se = software_engineer['Categoria IMC'].value_counts(normalize = True).to_frame()
bmi_se.columns = ['Proporção']
bmi_se.style.format({'Proporção': '{:.2%}'})

,Proporção
Categoria IMC,
Peso normal,50.00%
Sobrepeso,25.00%
Obesidade,25.00%


#4
##Método isin: Profissões e média

In [ ]:
professions = df['Profissão'].unique()
professions

array(['Eng. de Software', 'Médico(a)', 'Representante de Vendas',
       'Professor(a)', 'Enfermeiro(a)', 'Engenheiro(a)', 'Contador(a)',
       'Cientista', 'Advogado(a)', 'Pessoa Vendendora', 'Gerente'],
      dtype=object)

In [ ]:
median_all = df['Duração do sono'].mean()
print(f'A média de sono geral é de {median_all:.2f} horas.')

median_attorney_and_sales_rep = df[df['Profissão'].isin(['Advogado(a)', 'Representante de Vendas'])]['Duração do sono'].mean()
print(f'Média de horas de sono de advogados e representantes de venda: {median_attorney_and_sales_rep:.2f} horas.')

A média de sono geral é de 7.13 horas.
Média de horas de sono de advogados e representantes de venda: 7.35 horas.


In [ ]:
df_at_sr = {
    'Categoria': ['Geral', 'Advogados e Rep. de Vendas'],
    'Média de Duração do Sono': [median_all, median_attorney_and_sales_rep]
}

result_df_at_sr = pd.DataFrame(df_at_sr)

style_result_df_at_sr = result_df_at_sr.style.format({'Média de Duração do Sono': '{:.2f}'})
style_result_df_at_sr

,Categoria,Média de Duração do Sono
0,Geral,7.13
1,Advogados e Rep. de Vendas,7.35


#5
##Método isin: Menor sono entre profissões

In [ ]:
median_nurse_medic = df[df['Profissão'].isin(['Enfermeiro(a)', 'Médico(a)'])]
median_sleep = median_nurse_medic.groupby('Profissão')['Duração do sono'].mean()

profession_less_sleep = median_sleep.idxmin()
less_sleep = median_sleep.min()

print(f'A profissão com menos horas de sono é {profession_less_sleep} com uma média de {less_sleep:.2f} horas.')

A profissão com menos horas de sono é Médico(a) com uma média de 6.97 horas.


In [ ]:
df_n_m = {
    'Profissão': median_sleep.index,
    'Média de Duração do Sono': median_sleep.values
}

result_df_n_m = pd.DataFrame(df_n_m)

style_result_df_n_m = result_df_n_m.style.format({'Média de Duração do Sono': '{:.2f}'})
style_result_df_n_m

,Profissão,Média de Duração do Sono
0,Enfermeiro(a),7.05
1,Médico(a),6.97


#6
##Subconjunto com colunas existentes

In [ ]:
subset_loc = df.loc[:,['Identificador', 'Gênero', 'Idade', 'Pressão sanguínea', 'Frequência cardíaca']]
subset_loc.head()

,Identificador,Gênero,Idade,Pressão sanguínea,Frequência cardíaca
0,1,Homem,27,126/83,77
1,2,Homem,28,125/80,75
2,3,Homem,28,125/80,75
3,4,Homem,28,140/90,85
4,5,Homem,28,140/90,85


In [ ]:
subset_df = df[['Identificador', 'Gênero', 'Idade', 'Pressão sanguínea', 'Frequência cardíaca']]
subset_df.head()

,Identificador,Gênero,Idade,Pressão sanguínea,Frequência cardíaca
0,1,Homem,27,126/83,77
1,2,Homem,28,125/80,75
2,3,Homem,28,125/80,75
3,4,Homem,28,140/90,85
4,5,Homem,28,140/90,85


#7
##Método value_counts: Menor frequencia

In [ ]:
df['Profissão'].value_counts()

,count
Profissão,
Enfermeiro(a),72
Médico(a),71
Engenheiro(a),63
Advogado(a),47
Professor(a),40
Contador(a),37
Pessoa Vendendora,32
Eng. de Software,4
Cientista,4


In [ ]:
name_least_frequent_profession = df['Profissão'].value_counts().idxmin()
count_least_frequent_profession = df['Profissão'].value_counts().min()
print(f'A profissão menos frequente é {name_least_frequent_profession}, com {count_least_frequent_profession} pessoa(s).')

A profissão menos frequente é Gerente, com 1 pessoa(s).


#8
##Maior média: Pressão sanguínea

In [ ]:
df[['Pressão sistólica', 'Pressão diastólica']] = df['Pressão sanguínea'].str.split('/', expand = True)

df['Pressão sistólica'] = pd.to_numeric(df['Pressão sistólica'], errors='coerce')
df['Pressão diastólica'] = pd.to_numeric(df['Pressão diastólica'], errors='coerce')

pressure_gender = df.groupby('Gênero')['Pressão sistólica'].mean()
df_pressure_gender = pressure_gender.apply(lambda x: f'{x:.2f}')

higher_pressure_gender = pressure_gender.idxmax()
higher_pressure = pressure_gender.max()

print(f'O gênero com a maior média de pressão sanguínea é {higher_pressure_gender} com uma média de {higher_pressure:.2f} mmHg.')

O gênero com a maior média de pressão sanguínea é Mulher com uma média de 130.15 mmHg.


In [ ]:
df_pressure_gender.to_frame()

,Pressão sistólica
Gênero,
Homem,126.94
Mulher,130.15


#9
##Moda: Predominância

In [ ]:
mode_sleep = df['Duração do sono'].mode()[0]
print(f'A moda da duração do sono é {mode_sleep} horas.')

if mode_sleep == 8:
  print('É predominante dormir 8 horas por dia, porque a moda é {mode_sleep} horas.')
else:
  print(f'A moda da duração do sono é {mode_sleep} horas, então não é predominante dormir 8 horas por dia.')

A moda da duração do sono é 7.2 horas.
A moda da duração do sono é 7.2 horas, então não é predominante dormir 8 horas por dia.


#10
##Média: Passos e frequência cardíaca

In [ ]:
greater_than_70 = df[df['Frequência cardíaca'] > 70]
less_than_70 = df[df['Frequência cardíaca'] < 70]

mean_greater_than_70 = greater_than_70['Passos diários'].mean()
mean_less_than_70 = less_than_70['Passos diários'].mean()

data_steps_mean = {
    'Frequência cardíaca': ['Maior que 70', 'Menor que 70'],
    'Média de Passos': [mean_greater_than_70, mean_less_than_70]
}

if mean_greater_than_70 > mean_less_than_70:
  diff_steps = (mean_greater_than_70 - mean_less_than_70)
  print(f'A diferença de passos de pessoas com frequência cardíaca maior que 70 em relação a frequência menor que 70 é de {diff_steps:.2f} passos a mais.')
else:
  diff_steps = (mean_less_than_70 - mean_greater_than_70)
  print(f'A diferença de passos de pessoas com frequência cardíaca menor que 70 em relação a frequência maior que 70 é de {diff_steps:.2f} passos a mais.')

A diferença de passos de pessoas com frequência cardíaca menor que 70 em relação a frequência maior que 70 é de 102.68 passos a mais.


In [ ]:
df_data_steps_mean = pd.DataFrame(data_steps_mean)
df_data_steps_mean.style.format({'Média de Passos': '{:.2f}'})

,Frequência cardíaca,Média de Passos
0,Maior que 70,6472.93
1,Menor que 70,6575.61
